**Important: This notebook will only work with fastai-0.7.x. Do not try to run any fastai-1.x code from this path in the repository because it will load fastai-0.7.x**

## Dying Gasp Detector for Juniper Switch


In one of our use cases in telecom industry, we have devices that do not have capability to inform the network management system when the power to the chassis failed.

I implemented a system using fastai-0.7.x for one customer. The following is the steps I took for a Juniper EX4200 Switch. Due to copy right issues, I have not included any images in the notebook. You should be able to replicate the results using the steps:

I first started with the following Juniper documentation. See the section on 'Chassis status LEDs'. In our use case, it is decided to use the 'SYS' LED. We will be detecting Green/Off in the 'SYS' LED.

https://www.juniper.net/documentation/en_US/release-independent/junos/topics/topic-map/ex4200-chassis.html#chassis-ex4200-status-leds

NOTE: I was able to use MacBook Pro (15-inch, 2019) 2.3 GHz Intel Core i9, 16 GB 2400 MHz DDR4, to train the model. It just took a long time (over 10 hrs...). So a GPU based approach is recommended.

In [ ]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# This file contains all the main external libs we'll use
from fastai.imports import *

In [ ]:
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

`PATH` is the path to your data. Create the directory `data/dyingGasp` and then `train` and `valid` under that direcxtory. Under each `train` and `valid`, create two more directories `on`, `off`.
Find Juniper EX4200 images. Look at the images and put them under appropriate directories. That is the most time consuming part. For this project, we use around 30 images of each power on/off for the training and 10 images for validation.

In [1]:
PATH = "data/dyingGasp/"
sz=224

Fastai library will assume that you have *train* and *valid* directories. It also assumes that each dir will have subdirs for each class you wish to recognize (in this case, power 'on' and 'off').

We're going to use a <b>pre-trained</b> <b>resnet34</b> ImageNet CNN (1.2 million images and 1000 classes) model as a starting point. https://github.com/KaimingHe/deep-residual-networks


In [ ]:
arch = resnet34
data = ImageClassifierData.from_paths(PATH, tfms=tfms_from_model(arch, sz))
learn = ConvLearner.pretrained(arch, data, precompute=True)
learn.fit(0.01, 2)

In [ ]:
# this gives prediction for validation set. Predictions are in log scale
log_preds = learn.predict()

In [ ]:
preds = np.argmax(log_preds, axis=1)  # from log probabilities to 0 or 1
probs = np.exp(log_preds[:,1])        # pr(off)

In [ ]:
learn = ConvLearner.pretrained(arch, data, precompute=True)

In [ ]:
lrf=learn.lr_find()

The `learn` object contains an attribute `sched` that contains  learning rate scheduler, plot and find the learning rate visually.

In [ ]:
learn.sched.plot()

In our use case, choosing learning rate of 0.01 was Ok, so using 0.01.

In [ ]:
tfms = tfms_from_model(resnet34, sz, aug_tfms=transforms_side_on, max_zoom=1.1)

In [ ]:
data = ImageClassifierData.from_paths(PATH, tfms=tfms)
learn = ConvLearner.pretrained(arch, data, precompute=True)

In [ ]:
learn.fit(1e-2, 1)

In [ ]:
learn.precompute=False

In [ ]:
learn.fit(1e-2, 3, cycle_len=1)

Since we've got a pretty good model at this point, we might want to save it so we can load it again later without training it from scratch.

In [ ]:
learn.save('224_lastlayer')

In [ ]:
learn.load('224_lastlayer')

In [ ]:
learn.unfreeze()

In [ ]:
lr=np.array([1e-4,1e-3,1e-2])

In [ ]:
learn.fit(lr, 3, cycle_len=1, cycle_mult=2)

In [ ]:
learn.save('224_all')

In [ ]:
learn.load('224_all')

Using *test time augmentation* *TTA*.

In [ ]:
log_preds,y = learn.TTA()
probs = np.mean(np.exp(log_preds),0)

## In our use case, we are just following the steps suggested by fastai

1. precompute=True
1. Use `lr_find()` to find highest learning rate where loss is still clearly improving
1. Train last layer from precomputed activations for 1-2 epochs
1. Train last layer with data augmentation (i.e. precompute=False) for 2-3 epochs with cycle_len=1
1. Unfreeze all layers
1. Set earlier layers to 3x-10x lower learning rate than next higher layer
1. Use `lr_find()` again
1. Train full network with cycle_mult=2 until over-fitting